In [ ]:
cd /content/drive/MyDrive/textGCN-code/G1_Code - Copy

/content/drive/MyDrive/textGCN-code/G1_Code - Copy


In [ ]:

import numpy as np
import pandas as pd
from sklearn import preprocessing
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.metrics import accuracy_score

# Data loading

In [ ]:
d = pd.read_csv('./data/R8.sbert_emb.csv', header = None)
# print(d.head())
docs = []
labels = []
f = open('./data/R8.txt', 'r', encoding="utf8")
lines = f.readlines()
for line in lines:
    temp = line.split("\t")
    labels.append(temp[2])
f.close()

f_labels = []
for l in labels:
  # f_labels.append(int(l.replace("\n","")))   # MR dataset
  f_labels.append(l.replace("\n",""))  # sameval dataset
# f_labels = np.array(labels)
# print(f_labels)
le = preprocessing.LabelEncoder()
n_labels = le.fit_transform(f_labels)
un_labels = np.unique(n_labels)
print((d.values).shape)
print(len(n_labels))
x_all = d.values
y_all = np.array(n_labels)

(7674, 384)
7674


In [ ]:
https://drive.google.com/drive/folders/10n_c-VplpXH6V_TMGa3XtBoQV08Jr2yk?usp=sharing



test_size = [ 0.1, 0.3333, 0.5, 0.65, 0.9]


for i in test_size:
  # split train into train and test ----> by default, data first shuffled and then divided
  X_train, X_test, y_train, y_test = train_test_split(x_all, y_all, test_size= i , random_state=1, stratify=y_all)
  print("Test size = ", i)
  print("Xtrain and Xtest size",X_train.shape, X_test.shape)
  print("ytrain and ytest size",y_train.shape, y_test.shape)

  
  # # #Basic ML models
  # from sklearn.linear_model import LogisticRegression
  # LR_model = LogisticRegression(max_iter= 200)
  # LR_model.fit(X_train, y_train)

  # LR_yhat = LR_model.predict(X_test)
  # LR_score = accuracy_score(y_test, LR_yhat)
  # print("Accuracy on movie review dataset using LR together: %.3f" % (LR_score*100))

  # from sklearn.svm import SVC
  # SVM_model = SVC()
  # SVM_model.fit(X_train, y_train)
  # svm_yhat = SVM_model.predict(X_test)
  # svm_score = accuracy_score(y_test, svm_yhat)
  # print("Accuracy on movie review dataset using SVM together: %.3f" % (svm_score*100))

  # from sklearn.neighbors import KNeighborsClassifier
  # neigh = KNeighborsClassifier(n_neighbors=3)
  # neigh.fit(X_train, y_train)
  # knn_yhat = neigh.predict(X_test)
  # knn_score = accuracy_score(knn_yhat, y_test)
  # print("Accuracy on movie review dataset using KNN together: %.3f" % (knn_score*100))


  # X_Train & Xtest both are being passed while training - LPA
  # cell in which train-test both are being passed for training
  x_train_test = np.concatenate((X_train, X_test))
  no_label_test = [-1 for _ in range(len(y_test))]
  y_train_test = np.concatenate((y_train, no_label_test))

  model_all = LabelPropagation(gamma = 10, kernel = 'rbf')
  # model_all = LabelPropagation(max_iter=2000)
  model_all.fit(x_train_test, y_train_test)
  
  y_pred = model_all.predict(X_test)
  score = accuracy_score(y_test, y_pred)
  print("Accuracy on movie review dataset from LPA using all data: %.3f" % (score*100))

  # combining semi-supervised and supervised both, use predicted labels in training from LPA in supervised algorithm
  tran_label = model_all.transduction_
  # print(tran_label)

  from sklearn.linear_model import LogisticRegression
  model_all_LR = LogisticRegression(max_iter= 200)
  model_all_LR.fit(x_train_test, tran_label)

  y_pred_LR = model_all_LR.predict(X_test)
  score_LR = accuracy_score(y_test, y_pred_LR)
  print("Accuracy on movie review dataset using LPA and LR together: %.3f" % (score_LR*100))

  from sklearn.svm import SVC
  model_all_svm = SVC()
  model_all_svm.fit(x_train_test, tran_label)
  y_pred_svm = model_all_svm.predict(X_test)
  score_svm = accuracy_score(y_test, y_pred_svm)
  print("Accuracy on movie review dataset using LPA and SVM together: %.3f" % (score_svm*100))

  

Test size =  0.1
Xtrain and Xtest size (6906, 384) (768, 384)
ytrain and ytest size (6906,) (768,)
Accuracy on movie review dataset from LPA using all data: 94.010
Accuracy on movie review dataset using LPA and LR together: 95.703
Accuracy on movie review dataset using LPA and SVM together: 95.443
Test size =  0.3333
Xtrain and Xtest size (5116, 384) (2558, 384)
ytrain and ytest size (5116,) (2558,)
